# Analysis of output files
## Prepare environment, functions etc.

In [1]:
import os
import sys

# Add local src directory to the path. Then we are able to import our files.
module_path = os.path.abspath(os.path.join('src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas
import matplotlib
import numpy
from helpers import get_analysis_files, getFiles, FilePair
from myDataClasses import AnalysisFile

# Zapnout zobrazování grafů (procento uvozuje „magickou” zkratku IPythonu):
%matplotlib inline

path = 'analysisOutput/results'

In [3]:
#pandas.set_option('display.max_rows', None)
#pandas.read_csv?

In [4]:
# Important functions

def remove_bag_results(table):
    delim_index = list(table.iloc[0]).index("|")
    
    return table.iloc[:, 0:delim_index]

def get_cols_list(path: str):
    cols = pandas.read_csv(path, index_col=None, delimiter=" ", header=None)
    cols = remove_bag_results(cols)
    return list(cols.iloc[0])

def load_analysis_files(folder_path: str, column_list):
    files: AnalysisFile = get_analysis_files(folder_path)
    output_table = None
    
    for file in files:
        curr_table = pandas.read_csv(file.full_path, index_col=None, delimiter=" ", header=None)
        curr_table = remove_bag_results(curr_table)
        curr_table.columns = column_list
        curr_table["dataset"] = file.dataset
        curr_table["strategy"] = file.strategy
        
        if output_table is not None:
            output_table = output_table.append(curr_table, ignore_index=True)
        else:
            output_table = curr_table
    
    output_table = output_table.set_index(['strategy', 'dataset', 'id', "item_count"])
    return output_table

def construct_table_from(filePair: FilePair):
    solution_table = pandas.read_csv(filePair.solutionFile, header=None, index_col=None, delimiter=" ")
    data_table = pandas.read_csv(filePair.dataFile, header=None, index_col=None, delimiter=" ")
    
    item_count = data_table.iloc[0, 1]
    
    solution_table = solution_table.drop_duplicates(subset=[0], keep='first').reset_index()

    data_table = data_table.iloc[:, 4:]
    data_table = data_table[data_table.columns[::2]]

    info_table = pandas.concat([solution_table.iloc[:, 1], solution_table.iloc[:, 3], data_table.max(axis=1)], axis=1)
    info_table.columns = ["id", "best_value", "max_cost"]
    info_table["item_count"] = item_count
    return info_table

def get_info_from_datafiles(path: str):
    dataset = path.split("/")[-1]
    output_table = None
    for filePair in getFiles(path):
        curr_table = construct_table_from(filePair)
        curr_table["dataset"] = dataset
        curr_table = curr_table.set_index(["dataset", "item_count", "id"])
        
        if output_table is not None:
            output_table = output_table.append(curr_table)
        else:
            output_table = curr_table
    
    return output_table

## Put data from all analysis files into tables

In [5]:
# Create column lists

dp_cols = get_cols_list(f'{path}/DP/column_description_DP.dat')
dpweight_cols = get_cols_list(f'{path}/DPWeight/column_description_DPWeight.dat')
greedy_cols = get_cols_list(f'{path}/Greedy/column_description_Greedy.dat')
greedyone_cols = get_cols_list(f'{path}/GreedyOne/column_description_GreedyOne.dat')
fptas_cols = get_cols_list(f'{path}/FPTAS/column_description_FPTAS.dat')

In [6]:
# Load tables of all strategies
dp_table = load_analysis_files(f'{path}/DP', dp_cols)
dpweight_table = load_analysis_files(f'{path}/DPWeight', dpweight_cols)
greedy_table = load_analysis_files(f'{path}/Greedy', greedy_cols)
greedyone_table = load_analysis_files(f'{path}/GreedyOne', greedyone_cols)
fptas_table = load_analysis_files(f'{path}/FPTAS', fptas_cols)

## Get average time values for all strategies

In [7]:
# Add all table rows into 1 table
avg_times = fptas_table.drop(["relative_error"], axis=1).append(dp_table).append(dpweight_table).append(greedy_table).append(greedyone_table)

# Create a table of average times according to strategy and item_count columns
avg_times = avg_times.groupby(["strategy", "item_count"])["time"].mean().reset_index().set_index(["strategy", "item_count"])
avg_times = avg_times.round(4)

# Move all values of strategy column into separate columns
avg_times = avg_times.unstack("strategy")
avg_times.columns = avg_times.columns.droplevel()

# Save the dataframe to csv
avg_times.to_excel('analysisOutput/avg_times.xlsx', header=True)

avg_times

strategy,DP,DPWeight,FPTAS,Greedy,GreedyOne
item_count,,,,,
4,93.6982,1.4304,0.8519,0.0105,0.0079
10,456.8244,7.7131,10.2270,0.0160,0.0099
15,1106.8275,21.3662,32.6015,0.0204,0.0119
20,1950.3325,37.8990,83.6230,0.0269,0.0133
22,1876.8124,51.3970,116.6782,0.0317,0.0171
25,3221.8843,61.1677,175.6165,0.0305,0.0163
27,2785.1095,72.0347,222.2353,0.0372,0.0168
30,3349.9526,85.1810,292.5677,0.0370,0.0186
32,4461.4599,103.9543,331.5438,0.0387,0.0179


## Relative error/item_count analysis

In [8]:
# Create info_table which contains best values and highest cost for all instances
nk_info = get_info_from_datafiles("data/NK")
zkc_info = get_info_from_datafiles("data/ZKC")
zkw_info = get_info_from_datafiles("data/ZKW")

info_table = nk_info.append(zkc_info).append(zkw_info)

### Greedy and GreedyOne

In [9]:
greedy_analysis_table = greedy_table.reset_index().set_index(["dataset", "item_count", "id"])
greedyone_analysis_table = greedyone_table.reset_index().set_index(["dataset", "item_count", "id"])

error_analysis = greedy_analysis_table.append(greedyone_analysis_table).join(info_table)
error_analysis.rename(columns={'maximum_sum':'best_found_value'}, inplace=True)
error_analysis.drop(columns="max_cost", inplace=True)

error_analysis = error_analysis.reset_index().set_index(["strategy", "dataset", "item_count", "id"])
error_analysis.sort_values(by=["strategy", "dataset", "item_count", "id"], inplace=True)
error_analysis["measured_relative_error"] = numpy.abs(error_analysis["best_value"] - error_analysis["best_found_value"])/error_analysis["best_value"]
error_analysis.fillna(0.0, inplace=True)
error_analysis

best_found_value      time  best_value  \
strategy  dataset item_count id                                             
Greedy    NK      4          1                 1129  0.012579        1129   
                             2                 2966  0.011403        2966   
                             3                    0  0.010307           0   
                             4                   70  0.013882          70   
                             5                 2528  0.011245        2758   
...                                             ...       ...         ...   
GreedyOne ZKW     40         1729              4725  0.015815        5048   
                             1846              7956  0.015716        7956   
                             1935              7609  0.015919        7609   
                             1987              8712  0.015642        8712   
                             1991              4660  0.015857        4660   

                                   measured_relative_error  
strategy  dataset item_count id                             
Greedy    NK      4          1                    0.000000  
                             2                    0.000000  
                             3                    0.000000  
                             4                    0.000000  
                             5                    0.083394  
...                                                    ...  
GreedyOne ZKW     40         1729                 0.063986  
                             1846                 0.000000  
                             1935                 0.000000  
                             1987                 0.000000  
                             1991                 0.000000  

[27672 rows x 4 columns]

In [53]:
# Create Greedy/GreedyOne comparisons table
error_group = error_analysis.groupby(["strategy", "item_count"])["measured_relative_error"]

error_max = error_group.max().reset_index().set_index(["strategy", "item_count"]).rename(columns={'measured_relative_error':'max_measured_relative_error'})
error_avg = error_group.mean().reset_index().set_index(["strategy", "item_count"]).rename(columns={'measured_relative_error':'avg_measured_relative_error'})

# Construct, unstack
greedies_comparison = error_max.join(error_avg).round(6).unstack("strategy")
greedies_comparison.columns = ["Greedy_max_error", "GreedyOne_max_error", "Greedy_avg_error", "GreedyOne_avg_error"]

greedies_comparison.to_excel("analysisOutput/greedies_comparison.xlsx")

greedies_comparison


,Greedy_max_error,GreedyOne_max_error,Greedy_avg_error,GreedyOne_avg_error
item_count,,,,
4,0.994220,0.663519,0.201841,0.175520
10,0.980897,0.868900,0.094145,0.542395
15,0.911900,0.906724,0.057076,0.692689
20,0.870702,0.925672,0.041210,0.789942
22,0.986863,0.934436,0.040630,0.797961
25,0.882412,0.945159,0.033858,0.828923
27,0.869567,0.945211,0.031298,0.842392
30,0.682060,0.943471,0.025137,0.856279
32,0.916667,0.952653,0.024302,0.870382


### FPTAS

In [36]:
# Prepare the table for fptas analysis which has data from info_table
fptas_analysis_table = fptas_table.reset_index().drop(columns="strategy")
fptas_analysis_table = fptas_analysis_table.set_index(["dataset", "item_count", "relative_error", "id"])
fptas_analysis_table = fptas_analysis_table.join(info_table)
fptas_analysis_table.rename(columns={'maximum_sum':'best_found_value'}, inplace=True)

# Rearrange columns
fptas_analysis_table = fptas_analysis_table[["max_cost", "best_value", "best_found_value", "time"]]

# Add column with the measured relative errors
fptas_analysis_table["measured_relative_error"] = numpy.abs(fptas_analysis_table["best_value"] - fptas_analysis_table["best_found_value"])/fptas_analysis_table["best_value"]
fptas_analysis_table.fillna(0.0, inplace=True)

# Remove all rows where error == 100 %. These are statistical errors and there aren't many of them (< 25).
anomalies = fptas_analysis_table.query("measured_relative_error == 1.0").shape[0]
print(f'anomalies/all_values: {anomalies}/{fptas_analysis_table.shape[0]}')
fptas_analysis_table = fptas_analysis_table.query("measured_relative_error != 1.0")

fptas_analysis_table

anomalies/all_values: 21/69180


max_cost  best_value  \
dataset item_count id   relative_error                         
NK      4          1    0.2                 2084        1129   
                        0.4                 2084        1129   
                        0.3                 2084        1129   
                        0.1                 2084        1129   
                        0.5                 2084        1129   
...                                          ...         ...   
ZKW     40         1991 0.2                 4660        4660   
                        0.4                 4660        4660   
                        0.1                 4660        4660   
                        0.3                 4660        4660   
                        0.5                 4660        4660   

                                        best_found_value        time  \
dataset item_count id   relative_error                                 
NK      4          1    0.2                         1129    0.880371   
                        0.4                         1129    0.491997   
                        0.3                         1129    0.832692   
                        0.1                         1129    0.871974   
                        0.5                         1129    0.419170   
...                                                  ...         ...   
ZKW     40         1991 0.2                         4660  325.573507   
                        0.4                         4660  211.777098   
                        0.1                         4660  650.592674   
                        0.3                         4660  181.874641   
                        0.5                         4660  149.923201   

                                        measured_relative_error  
dataset item_count id   relative_error                           
NK      4          1    0.2                                 0.0  
                        0.4                                 0.0  
                        0.3                                 0.0  
                        0.1                                 0.0  
                        0.5                                 0.0  
...                                                         ...  
ZKW     40         1991 0.2                                 0.0  
                        0.4                                 0.0  
                        0.1                                 0.0  
                        0.3                                 0.0  
                        0.5                                 0.0  

[69159 rows x 5 columns]

In [37]:
#fptas_analysis_table["ommited_bits"] = (fptas_analysis_table.index.get_level_values("relative_error") * fptas_analysis_table["max_cost"])/fptas_analysis_table.index.get_level_values("item_count")
#fptas_analysis_table["ommited_bits"] = numpy.log2(fptas_analysis_table["ommited_bits"])

#fptas_analysis_table["actual_max_relative_error"] = (fptas_analysis_table.index.get_level_values("item_count") * numpy.power(2, numpy.floor(fptas_analysis_table["ommited_bits"])))/fptas_analysis_table["max_cost"]

In [38]:
# Create the aggregated table

error_group = fptas_analysis_table.groupby(["item_count", "relative_error"])["measured_relative_error"]
time_group = fptas_analysis_table.groupby(["item_count", "relative_error"])["time"]

error_max = error_group.max().reset_index().set_index(["item_count", "relative_error"]).rename(columns={'measured_relative_error':'max_measured_relative_error'})
error_avg = error_group.mean().reset_index().set_index(["item_count", "relative_error"]).rename(columns={'measured_relative_error':'avg_measured_relative_error'})
time_avg = time_group.mean().reset_index().set_index(["item_count", "relative_error"])

time_avg["time"] = time_avg["time"]
time_avg.rename(columns={'time':'avg_time[ms]'}, inplace=True)

fptas_error_time = error_max.join(error_avg).join(time_avg)
fptas_error_time = fptas_error_time.round(6)

fptas_error_time.to_excel("analysisOutput/fptas_error_time.xlsx")

fptas_error_time

max_measured_relative_error  \
item_count relative_error                                
4          0.1                                0.021425   
           0.2                                0.197895   
           0.3                                0.197895   
           0.4                                0.197895   
           0.5                                0.258182   
10         0.1                                0.016220   
           0.2                                0.078431   
           0.3                                0.078431   
           0.4                                0.078431   
           0.5                                0.102442   
15         0.1                                0.009777   
           0.2                                0.013001   
           0.3                                0.064089   
           0.4                                0.064089   
           0.5                                0.064089   
20         0.1                                0.001941   
           0.2                                0.017655   
           0.3                                0.017655   
           0.4                                0.026297   
           0.5                                0.079371   
22         0.1                                0.007236   
           0.2                                0.007236   
           0.3                                0.016510   
           0.4                                0.016510   
           0.5                                0.019069   
25         0.1                                0.000850   
           0.2                                0.017145   
           0.3                                0.017145   
           0.4                                0.017690   
           0.5                                0.017145   
27         0.1                                0.000708   
           0.2                                0.003119   
           0.3                                0.005409   
           0.4                                0.005075   
           0.5                                0.013829   
30         0.1                                0.000643   
           0.2                                0.008142   
           0.3                                0.008142   
           0.4                                0.008142   
           0.5                                0.008142   
32         0.1                                0.000496   
           0.2                                0.001318   
           0.3                                0.001879   
           0.4                                0.003943   
           0.5                                0.020924   
35         0.1                                0.000455   
           0.2                                0.002092   
           0.3                                0.006166   
           0.4                                0.006166   
           0.5                                0.006166   
37         0.1                                0.000366   
           0.2                                0.004175   
           0.3                                0.008070   
           0.4                                0.008070   
           0.5                                0.008070   
40         0.1                                0.000407   
           0.2                                0.000908   
           0.3                                0.000915   
           0.4                                0.001882   
           0.5                                0.001882   

                           avg_measured_relative_error  avg_time[ms]  
item_count relative_error                                             
4          0.1                                0.000288      1.523912  
           0.2                                0.001096      1.039187  
           0.3                                0.001224      0.719200  
           0.4                                0.003718      0.558511  
    